Imports

In [101]:
import numpy as np
from bokeh.plotting import figure
from bokeh.models import LabelSet
import datetime as dt
import panel as pn
import concurrent.futures
from bokeh.models import ColumnDataSource
pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df
from datetime import datetime, timezone
import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d
from datetime import date, time
import random
import numpy as np
import pandas as pd



import hvplot.streamz
import holoviews as hv
from streamz import Stream
from streamz.dataframe import DataFrame

import panel as pn
from time import sleep
from bokeh.models.formatters import DatetimeTickFormatter
pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

### Configure cognite client

In [2]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"
# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"


from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter



### Define Dataset

In [3]:
#  
wellcom_dataset = "src:015:wellcom"
witsml_dataset = "src:014:witsmldata"
active_wellbore = "active:wellbore"

#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
#Get sensor data 
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)

### Convert to pandas dataframe and flatten

In [5]:
# Wellbores
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)

# Sensordata
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml.rename(columns ={'name': 'sensorName'} , inplace = True)
# Flatten Sensordata
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'] )], axis=1)

#Limiting columns
_pd_witsml_flattened_lim = _pd_witsml_flattened[['id', 'uidWell', 'sensorName', 'name', 'uidWellbore',  'nameWellbore','external_id',  'is_string', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType',  'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid']]





In [6]:

#Get name and id of wellbore
pd_wellbores = _pd_witsml_flattened[['nameWellbore', 'uidWellbore']]
#Get rid of duplicates
pd_wellbores = pd_wellbores.drop_duplicates(subset=['uidWellbore'])
#Create dictionary 
wellbore_map = {row['nameWellbore']: row['uidWellbore'] for index, row in pd_wellbores.iterrows()}

#Get name and id of sensor
pd_sensors = _pd_witsml_flattened[['sensorName', 'id', 'external_id']]
#Get rid of duplicates
pd_sensors = pd_sensors.drop_duplicates(subset=['id'])
 

# Main Dashboard

In [113]:

pn.param.ParamMethod.loading_indicator = True
# Uncomment for dark mode
# pn.config.theme = 'dark'

# Defining the paramterised class 
class CustomDashboard(param.Parameterized):
    
    
    #Parameter for wellbore selector 
    wellbore = param.ObjectSelector(default=pd_wellbores['nameWellbore'].iloc[0], objects=list( sorted(pd_wellbores['nameWellbore'])    ))
    #Dropdown for senors
    sensor = param.ObjectSelector(default= _pd_witsml_flattened[_pd_witsml_flattened['nameWellbore'] == pd_wellbores['nameWellbore'].iloc[0] ]['sensorName'].iloc[0]   , objects=list( _pd_witsml_flattened[_pd_witsml_flattened['nameWellbore'] == pd_wellbores['nameWellbore'].iloc[0] ]['sensorName']) ) 
    
    live_mode  = param.Boolean(label = 'Live mode', default = False )
    #This is a checkbox that allows the user to view all sensor data, off by default because it's very slow
    show_all_dates = param.Boolean(label = 'Show all dates (makes app slower)', default = False )
    #Date range to define what range to look for sensor data in           
    date_range = param.DateRange(default=(date.today(), date.today() + timedelta(days=1)))  # Changed to DateRange
  


    @param.depends('date_range', watch=True)
    def date_range_test(self):
        start_date, end_date = self.date_range
        print(f"Start Date: {start_date}, End Date: {end_date}")


    @param.depends('wellbore', watch=True)
    def update_sensors(self):
        wellbore_id = wellbore_map[self.wellbore]
        filtered_sensors = _pd_witsml_flattened[_pd_witsml_flattened['uidWellbore'] == wellbore_id]
        sensor_names = list(sorted(filtered_sensors['sensorName']))
        self.param['sensor'].objects = sensor_names
        if sensor_names:
                self.sensor = sensor_names[0]
        else:
            self.sensor= None

    
    
    def get_sensor_id(self):
        wellbore_id= wellbore_map[self.wellbore]
        sensor_data = _pd_witsml_flattened[(_pd_witsml_flattened['uidWellbore'] == wellbore_id) & (_pd_witsml_flattened['sensorName'] == self.sensor)]
        if not sensor_data.empty:
            self.sensorid = sensor_data['id'].iloc[0]
            return sensor_data['id'].iloc[0]
        return None

    
    @param.depends('sensor')
    def update_sensor_label(self):
        sensorid = self.get_sensor_id()
        if sensorid:
            return pn.pane.Str(f'Selected Sensor ID: {sensorid}')
        return pn.pane.Str('No sensor selected')
    


    def emit(self):
        try:
            # df_new = pd.DataFrame({'a': np.random.randint(1, 10, 1)}, index=[dt.datetime.now()])
            
            df_new = client.time_series.data.retrieve(id=368728207134017,start=_glb_datetime,  end=MAX_TIMESTAMP_MS)#ColumnDataSource({"x": np.arange(10), "y": np.arange(10)})
            df_new = df_new.to_pandas().sort_index(ascending=False)  
            df_new['Value'] = df_new[df_new.columns[0]]
            df_new['Date'] = pd.to_datetime(df_new.index)

            print("Hehe new line ")

            # make new datetime
            last_time = df_new['Date'].max() # Update the last timestam
            dt_obj = dt.datetime.strptime(str(last_time), '%Y-%m-%d %H:%M:%S')
            year = dt_obj.year
            month = dt_obj.month
            day = dt_obj.day
            hour = dt_obj.hour
            minute = dt_obj.minute
            second = dt_obj.second
            new_time = dt.datetime(year, month, day, hour, minute, second, tzinfo=timezone.utc)
            # try:
            #     (new_time > timesss)
            #     print(f" new_time {new_time}, timesss {timesss}")
            # except Exception as e:
            #     print(f"error : {e}")

            if (new_time > _glb_datetime):

            # if (len(stream.to_dataframe()) > 5):
            #     print("Stopping now")
                print(df_new)
                self.df_stream.emit(df_new)
                print(f"new_time is {new_time} while _glb_datetime is {_glb_datetime}")
                _glb_datetime = new_time

        except Exception as e:
            print(f"error {e}")
        finally:
            print("Step finished.")  

    
    
    #Main function for plotting. Too long, should be refactored with more functions
    @param.depends('sensor', 'date_range' , 'show_all_dates', 'live_mode')
    def plot(self):

        start_date_output, end_date_output = self.date_range

        if self.live_mode:

            # Init stream
            self.stream = Stream()
            _glb_datetime = dt.datetime(2024, 8, 5, 00, 00, 00, tzinfo=timezone.utc)
            df_ = client.time_series.data.retrieve(id=368728207134017,start=_glb_datetime,  end=MAX_TIMESTAMP_MS)#ColumnDataSource({"x": np.arange(10), "y": np.arange(10)})
            df_ = df_.to_pandas().sort_index(ascending=False)  
            df_['Value'] = df_[df_.columns[0]]
            df_['Date'] = pd.to_datetime(df_.index)
            self.df_stream = DataFrame(self.stream, example=df_)        
            
            start_date, end_date = self.date_range
            min_datetime = datetime.combine(start_date, time.min)
            min_timestamp =min_datetime.timestamp()
            df_ = client.time_series.data.retrieve(id=368728207134017,start=min_timestamp,  end=MAX_TIMESTAMP_MS)
            df_ = df_.to_pandas().sort_index(ascending=False)  
            df_['Value'] = df_[df_.columns[0]]
            df_['Date'] = pd.to_datetime(df_.index)
            self.df_stream = DataFrame(self.stream, example=df_)
            plot_test = self.df_stream.hvplot(kind='scatter',x='Value', y ='Date')


            return plot_test
        
        

        if self.sensor:

            if(self.show_all_dates):
                print("Show all dates is now checked and timestamps are set to cognite utils.") 
                print(self.show_all_dates)
                min_timestamps_ms =MIN_TIMESTAMP_MS
                max_timestamps_ms = MAX_TIMESTAMP_MS
            #If user is not viewing all the dates, then min and max should be min and max of 24 hours from the selected date.    
            else:
                print("Show all dates is set to false and timestamps are set to selected date") 
                #Getting time stamps max and min of 24hrs of selected date
                #Keeping datetime as well as timestamps, datetime is used later when filtering out operations data
                min_datetime = datetime.combine(start_date_output, time.min)
                min_timestamp =min_datetime.timestamp()
                max_datetime = datetime.combine((end_date_output + timedelta(hours = 24)) , time.min)
                max_timestamp = max_datetime.timestamp()
                #Need to be milliseconds to work with cognite
                min_timestamps_ms = int(min_timestamp *1000 )
                max_timestamps_ms = int(  max_timestamp * 1000)
                
            
            #Getting sensor id
            sensor_id = int(self.get_sensor_id())
            sensor_data = client.time_series.data.retrieve(id=sensor_id,start=min_timestamps_ms,  end=max_timestamps_ms + 1)
            
            #Converting sensor data to pandas
            sensor_data_to_pd = sensor_data.to_pandas()
            # Reindexing the sensor data
            sensor_data_to_pd['Value'] = sensor_data_to_pd[sensor_data_to_pd.columns]
            sensor_data_to_pd['Date'] = pd.to_datetime(sensor_data_to_pd.index)
            #Adding a more consistant series of time stamps
            
            #Skips this part if  user is not viewing all date
            if (self.show_all_dates):
                start_date = sensor_data_to_pd['Date'].min()
                end_date = sensor_data_to_pd['Date'].max()
                #Finding the smallest interval to define the frequency
            else:
                 start_date = start_date_output
                 end_date =end_date_output
                 print("This should run. Have a look at start date and end date:")
                 print(start_date)
                 print(end_date)
            
            time_diff = sensor_data_to_pd['Date'].diff().min()         



            if(pd.isnull (time_diff)):
                 print("There is no time diff, setting frequency to seconds")
                 time_diff = 'S'
            else:
                print("debug else")
                print(time_diff)     

            #Generate timestamps
            new_timestamps = pd.date_range(start=start_date, end=end_date, freq= time_diff)

            
            print("Let's check the smallest generated timestamp: " )
            print(new_timestamps.min)
            
            #Create a new DataFrame with the new timestamps
            new_data = pd.DataFrame(new_timestamps, columns=['New_Timestamps'])
            #Merge the new data with the original data using an outer join
            sensor_data_to_pd = pd.merge(sensor_data_to_pd, new_data, how='outer', left_on='Date', right_on='New_Timestamps')
            #Joining the new timestamps 
            sensor_data_to_pd.set_index('New_Timestamps')             

            if not sensor_data_to_pd.empty:
                

                
                max_sensor_value = sensor_data_to_pd['Value'].max()
                min_sensor_value = sensor_data_to_pd['Value'].min()
                center =sensor_data_to_pd['Value'].median()
                sensor_plot = sensor_data_to_pd.hvplot.line(x='Value', y='New_Timestamps', autorange="y", title=f'Sensor Data for {self.sensor}').opts(invert_yaxis=True, width = 800, height = 1000)
                uptime_downtime_data = [
                    {'Start': '2022-11-25 05:00:00', 'End': '2022-11-25 05:30:00', 'Status': 'Uptime'},
                    {'Start': '2022-11-25 06:12:30', 'End': '2022-11-25 06:45:30', 'Status': 'Downtime'},
                    {'Start': '2022-11-25 08:12:30', 'End': '2022-11-25 10:12:30', 'Status': 'Uptime'},

                    {'Start': '2022-11-28 05:00:00', 'End': '2022-11-28 05:30:00', 'Status': 'Downtime'},
                    {'Start': '2022-11-28 06:12:30', 'End': '2022-11-28 06:45:30', 'Status': 'Uptime'},
                    {'Start': '2022-11-28 08:12:30', 'End': '2022-11-28 10:12:30', 'Status': 'Uptime'},
                    # Add more uptime/downtime periods as needed
                ]

                

                uptime_downtime_df = pd.DataFrame(uptime_downtime_data)

                uptime_downtime_df['Start'] = pd.to_datetime(uptime_downtime_df['Start'])
                uptime_downtime_df['End'] = pd.to_datetime(uptime_downtime_df['End'])


                  
                if(not(self.show_all_dates)):
                    #Filtering out the uptime_downtime data to remove data that is no within the range of the selected date
                    uptime_downtime_df = uptime_downtime_df[(uptime_downtime_df['Start'] >= min_datetime) & (uptime_downtime_df['End'] <= max_datetime)]
                
                labels = []
                rects = []
                for _, row in uptime_downtime_df.iterrows():
                    midpoint = row['Start'] + (row['End'] - row['Start']) /2
                    labels.append(hv.Text( center, midpoint, row['Status'] ).opts(text_font_style = 'bold', text_align = 'left') )
                    color = 'green' if row['Status'] == 'Uptime' else 'red'
                    rects.append(hv.Rectangles((min_sensor_value,row['Start'],max_sensor_value , row['End'])).opts(color=color, alpha=0.2))
                
                rects_overlay = hv.Overlay(rects)
                labels_overlay = hv.Overlay(labels)
        
                return  sensor_plot *rects_overlay * labels_overlay
        
        
        return pn.pane.Str('No data available for the selected sensor')

    
    def yesClick(self, event= None): 
        print('Clicked yes')
   
    def noClick(self, event= None): 
        print('Clicked no')
        self.template.close_modal()
        
    def show_popup(self, event= None):
        self.template.open_modal()


    #Run update sensor method to populate sensor dropdown
    def __init__(self, **params):
        super().__init__(**params)

        




        self.template = pn.template.BootstrapTemplate(title='Invisible lost time Dashboard')
      
        self.show_popup_action = pn.widgets.Button(name= "Show Popup", button_type='primary' )
        self.show_popup_action.on_click(self.show_popup)
        # self.template.modal(align='center')
        self.template.modal.append("## This is a modal")
        self.yesButton = pn.widgets.Button(name= "Yes", button_type='primary' )
        self.noButton = pn.widgets.Button(name= "No", button_type='primary' )
        self.yesButton.on_click(self.yesClick)
        self.noButton.on_click(self.noClick)

        self.template.modal.append(self.yesButton) 
        self.template.modal.append(self.noButton) 
        self.template.sidebar.append(pn.Param(self.param, widgets={'date_range': {'type':pn.widgets.DateRangePicker} }  )  )
        self.template.sidebar.append(self.update_sensor_label)
        self.template.sidebar.append(self.show_popup_action)
    
        self.template.main.append(pn.Row(
            pn.Card(self.plot,  title= 'Sensor data')  )  )
        self.update_sensors()

        cb = pn.state.add_periodic_callback(self.emit, 5000, timeout=600000)
        # Set up streaming DataFrame


        
        

        
        
        
        


    def panel(self):

        return self.template

        

# Instantiate the dashboard
dashboard = CustomDashboard()

# Trigger initial sensor update
#dashboard.update_sensors()

# Display the dashboard
dashboard.panel().show()





Show all dates is set to false and timestamps are set to selected date
This should run. Have a look at start date and end date:
2024-08-07
2024-08-08
There is no time diff, setting frequency to seconds
Let's check the smallest generated timestamp: 
<bound method Index.min of DatetimeIndex(['2024-08-07 00:00:00', '2024-08-07 00:00:01',
               '2024-08-07 00:00:02', '2024-08-07 00:00:03',
               '2024-08-07 00:00:04', '2024-08-07 00:00:05',
               '2024-08-07 00:00:06', '2024-08-07 00:00:07',
               '2024-08-07 00:00:08', '2024-08-07 00:00:09',
               ...
               '2024-08-07 23:59:51', '2024-08-07 23:59:52',
               '2024-08-07 23:59:53', '2024-08-07 23:59:54',
               '2024-08-07 23:59:55', '2024-08-07 23:59:56',
               '2024-08-07 23:59:57', '2024-08-07 23:59:58',
               '2024-08-07 23:59:59', '2024-08-08 00:00:00'],
              dtype='datetime64[ns]', length=86401, freq='S')>
Show all dates is set to false 

C:\Users\mats.vaage\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\mats.vaage\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


error cannot access local variable '_glb_datetime' where it is not associated with a value
Step finished.
Clicked yes
Clicked no
error cannot access local variable '_glb_datetime' where it is not associated with a value
Step finished.
error cannot access local variable '_glb_datetime' where it is not associated with a value
Step finished.
error cannot access local variable '_glb_datetime' where it is not associated with a value
Step finished.
Start Date: 2022-11-24, End Date: 2022-11-28
Show all dates is set to false and timestamps are set to selected date
This should run. Have a look at start date and end date:
2022-11-24
2022-11-28
debug else
0 days 00:00:01
Let's check the smallest generated timestamp: 
<bound method Index.min of DatetimeIndex(['2022-11-24 00:00:00', '2022-11-24 00:00:01',
               '2022-11-24 00:00:02', '2022-11-24 00:00:03',
               '2022-11-24 00:00:04', '2022-11-24 00:00:05',
               '2022-11-24 00:00:06', '2022-11-24 00:00:07',
             